In [ ]:
import os
import requests

JIRA_URL = 'https://Hibernate.atlassian.net'
USERNAME = 'omar.khabthani123@gmail.com'
API_TOKEN = APIKEY
PROJECT_KEY = 'HHH'

issues_url = f"{JIRA_URL}/rest/api/2/search"

issue_url = f"{JIRA_URL}/rest/api/2/issue/{{}}"

attachment_url = f"{JIRA_URL}/secure/attachment/{{}}/{{}}"


auth = (USERNAME, API_TOKEN)

params = {
    'jql': f'project={PROJECT_KEY}',
    'fields': 'key',
    'maxResults': 50,  
    'startAt': 0  
}

output_directory = 'attachments'
os.makedirs(output_directory, exist_ok=True)

allowed_file_types = ['xml', 'json', 'txt', 'csv', 'xls', 'xlsx', 'pdf', 'yaml', 'msg' ,'log', 'sql' , 'zip' , 'docx','png','jpeg']

result_count = 0

while True:
    response = requests.get(issues_url, auth=auth, params=params)
    response.raise_for_status()


    data = response.json()

    issues = data.get('issues', [])

    if not issues or result_count >= 50:
        break

    for issue in issues:
        issue_key = issue['key']
        print(f"Processing issue: {issue_key}")

        response = requests.get(issue_url.format(issue_key), auth=auth)
        response.raise_for_status()

        issue_details = response.json()

        attachments = issue_details.get('fields', {}).get('attachment', [])

        supported_attachments_found = False

        for attachment in attachments:
            attachment_id = attachment.get('id')
            attachment_name = attachment.get('filename')
            attachment_file_type = attachment_name.split('.')[-1].lower() if attachment_name else None

            if attachment_id and attachment_name:
                if attachment_file_type in allowed_file_types:
                    attachment_response = requests.get(attachment_url.format(attachment_id, attachment_name), auth=auth)

                    if attachment_response.status_code == 200:
                        output_path = os.path.join(output_directory, f"{issue_key}_{attachment_name}")
                        with open(output_path, 'wb') as f:
                            f.write(attachment_response.content)
                            supported_attachments_found = True
                    else:
                        print(f"Failed to download attachment for issue {issue_key}, attachment ID {attachment_id}, status code: {attachment_response.status_code}")
                else:
                    print(f"Skipping unsupported attachment for issue {issue_key}: {attachment_name} (Type: {attachment_file_type})")

        if not supported_attachments_found:
            print(f"No supported attachments found for issue {issue_key}")

        result_count += 1

    params['startAt'] += 50

print("All issues processed.")

Processing issue: HHH-17785
No supported attachments found for issue HHH-17785
Processing issue: HHH-17784
No supported attachments found for issue HHH-17784
Processing issue: HHH-17783
Skipping unsupported attachment for issue HHH-17783: ORMUnitTestCase.java (Type: java)
No supported attachments found for issue HHH-17783
Processing issue: HHH-17782
No supported attachments found for issue HHH-17782
Processing issue: HHH-17781
No supported attachments found for issue HHH-17781
Processing issue: HHH-17780
No supported attachments found for issue HHH-17780
Processing issue: HHH-17779
No supported attachments found for issue HHH-17779
Processing issue: HHH-17778
No supported attachments found for issue HHH-17778
Processing issue: HHH-17777
No supported attachments found for issue HHH-17777
Processing issue: HHH-17776
No supported attachments found for issue HHH-17776
Processing issue: HHH-17775
No supported attachments found for issue HHH-17775
Processing issue: HHH-17774
No supported att

In [ ]:
import os
import requests

JIRA_URL = 'https://Hibernate.atlassian.net'
USERNAME = 'omar.khabthani123@gmail.com'
API_TOKEN = APIKEY

projects_url = f"{JIRA_URL}/rest/api/2/project"
issues_url = f"{JIRA_URL}/rest/api/2/search"
issue_url = f"{JIRA_URL}/rest/api/2/issue/{{}}"
attachment_url = f"{JIRA_URL}/secure/attachment/{{}}/{{}}"

auth = (USERNAME, API_TOKEN)

params = {
    'fields': 'key',
    'maxResults': 50,  
    'startAt': 0  
}

allowed_file_types = ['xml', 'json', 'txt', 'csv', 'xls', 'xlsx', 'pdf', 'yaml', 'msg', 'log', 'sql', 'zip', 'docx', 'png', 'jpeg']

result_count = 0

output_directory = 'attachments'
os.makedirs(output_directory, exist_ok=True)

response = requests.get(projects_url, auth=auth)
response.raise_for_status()
projects_data = response.json()

for project_data in projects_data:
    project_key = project_data['key']
    project_name = project_data['name']
    project_directory = os.path.join(output_directory, project_key)
    os.makedirs(project_directory, exist_ok=True)

    print(f"Processing project: {project_name} ({project_key})")

    params['jql'] = f'project={project_key}'

    while True:
        response = requests.get(issues_url, auth=auth, params=params)
        response.raise_for_status()

        data = response.json()

        issues = data.get('issues', [])

        if not issues or result_count >= 50:
            break

        for issue in issues:
            issue_key = issue['key']
            issue_directory = os.path.join(project_directory, issue_key)
            os.makedirs(issue_directory, exist_ok=True)

            print(f"Processing issue: {issue_key}")

            response = requests.get(issue_url.format(issue_key), auth=auth)
            response.raise_for_status()

            issue_details = response.json()

            attachments = issue_details.get('fields', {}).get('attachment', [])

            supported_attachments_found = False

            for attachment in attachments:
                attachment_id = attachment.get('id')
                attachment_name = attachment.get('filename')
                attachment_file_type = attachment_name.split('.')[-1].lower() if attachment_name else None

                if attachment_id and attachment_name:
                    if attachment_file_type in allowed_file_types:
                        attachment_response = requests.get(attachment_url.format(attachment_id, attachment_name), auth=auth)

                        if attachment_response.status_code == 200:
                            output_path = os.path.join(issue_directory, attachment_name)
                            with open(output_path, 'wb') as f:
                                f.write(attachment_response.content)
                                supported_attachments_found = True
                        else:
                            print(f"Failed to download attachment for issue {issue_key}, attachment ID {attachment_id}, status code: {attachment_response.status_code}")
                    else:
                        print(f"Skipping unsupported attachment for issue {issue_key}: {attachment_name} (Type: {attachment_file_type})")

            if not supported_attachments_found:
                print(f"No supported attachments found for issue {issue_key}")

            result_count += 1

        params['startAt'] += 50

    result_count = 0

print("All projects processed.")

Processing project: Hibernate Commons Annotations (HCANN)
Processing issue: HCANN-134
No supported attachments found for issue HCANN-134
Processing issue: HCANN-133
No supported attachments found for issue HCANN-133
Processing issue: HCANN-132
No supported attachments found for issue HCANN-132
Processing issue: HCANN-131
No supported attachments found for issue HCANN-131
Processing issue: HCANN-129
No supported attachments found for issue HCANN-129
Processing issue: HCANN-128
No supported attachments found for issue HCANN-128
Processing issue: HCANN-127
No supported attachments found for issue HCANN-127
Processing issue: HCANN-126
No supported attachments found for issue HCANN-126
Processing issue: HCANN-125
No supported attachments found for issue HCANN-125
Processing issue: HCANN-124
No supported attachments found for issue HCANN-124
Processing issue: HCANN-123
No supported attachments found for issue HCANN-123
Processing issue: HCANN-122
No supported attachments found for issue HCAN

No supported attachments found for issue ANN-607
Processing issue: ANN-606
No supported attachments found for issue ANN-606
Processing issue: ANN-605
Skipping unsupported attachment for issue ANN-605: Enumerated.patch (Type: patch)
No supported attachments found for issue ANN-605
Processing issue: ANN-603
Skipping unsupported attachment for issue ANN-603: ann-example_eager_join.patch (Type: patch)
No supported attachments found for issue ANN-603
Processing project: z - Hibernate Validator 3 (HVAL)
Processing project: Hibernate jDocBook Style (STYLE)
Processing project: Bean Validation (BVAL)
Processing issue: BVAL-657
No supported attachments found for issue BVAL-657
Processing issue: BVAL-656
No supported attachments found for issue BVAL-656
Processing issue: BVAL-655
No supported attachments found for issue BVAL-655
Processing issue: BVAL-653
No supported attachments found for issue BVAL-653
Processing issue: BVAL-652
No supported attachments found for issue BVAL-652
Processing issue

No supported attachments found for issue HSEARCH-4927
Processing issue: HSEARCH-4926
No supported attachments found for issue HSEARCH-4926
Processing issue: HSEARCH-4925
No supported attachments found for issue HSEARCH-4925
Processing issue: HSEARCH-4924
No supported attachments found for issue HSEARCH-4924
Processing issue: HSEARCH-4923
No supported attachments found for issue HSEARCH-4923
Processing issue: HSEARCH-4922
No supported attachments found for issue HSEARCH-4922
Processing issue: HSEARCH-4921
No supported attachments found for issue HSEARCH-4921
Processing issue: HSEARCH-4920
No supported attachments found for issue HSEARCH-4920
Processing issue: HSEARCH-4919
No supported attachments found for issue HSEARCH-4919
Processing issue: HSEARCH-4918
No supported attachments found for issue HSEARCH-4918
Processing issue: HSEARCH-4917
No supported attachments found for issue HSEARCH-4917
Processing issue: HSEARCH-4916
No supported attachments found for issue HSEARCH-4916
Processing 

Processing issue: HHH-17663
No supported attachments found for issue HHH-17663
Processing issue: HHH-17662
No supported attachments found for issue HHH-17662
Processing issue: HHH-17661
Processing issue: HHH-17660
No supported attachments found for issue HHH-17660
Processing issue: HHH-17659
No supported attachments found for issue HHH-17659
Processing issue: HHH-17658
No supported attachments found for issue HHH-17658
Processing issue: HHH-17657
No supported attachments found for issue HHH-17657
Processing issue: HHH-17656
No supported attachments found for issue HHH-17656
Processing issue: HHH-17655
Skipping unsupported attachment for issue HHH-17655: BooleanMappingTests.java (Type: java)
No supported attachments found for issue HHH-17655
Processing issue: HHH-17654
No supported attachments found for issue HHH-17654
Processing issue: HHH-17653
Processing issue: HHH-17652
No supported attachments found for issue HHH-17652
Processing issue: HHH-17651
Processing issue: HHH-17650
Skippin

No supported attachments found for issue HB-1139
Processing issue: HB-1138
No supported attachments found for issue HB-1138
Processing issue: HB-1137
No supported attachments found for issue HB-1137
Processing issue: HB-1136
Skipping unsupported attachment for issue HB-1136: EntityPersister.patch (Type: patch)
No supported attachments found for issue HB-1136
Processing issue: HB-1135
No supported attachments found for issue HB-1135
Processing issue: HB-1134
No supported attachments found for issue HB-1134
Processing issue: HB-1133
No supported attachments found for issue HB-1133
Processing issue: HB-1132
Skipping unsupported attachment for issue HB-1132: CSVFile.java (Type: java)
Skipping unsupported attachment for issue HB-1132: Record.java (Type: java)
Processing issue: HB-1131
No supported attachments found for issue HB-1131
Processing issue: HB-1130
No supported attachments found for issue HB-1130
Processing issue: HB-1129
No supported attachments found for issue HB-1129
Processing

No supported attachments found for issue WEBSITE-155
Processing issue: WEBSITE-154
No supported attachments found for issue WEBSITE-154
Processing project: Hibernate Semantic Query (SQM)
Processing project: z- Hibernate Metamodel Generator (METAGEN)
All projects processed.


In [1]:
import os
import requests

JIRA_URL = 'https://Hibernate.atlassian.net'
USERNAME = 'omar.khabthani123@gmail.com'
API_TOKEN = 'ATATT3xFfGF0x25O0JFpP7xgI7XhKTEyCGqU1rZgzkZrcmmrhbefD5w3ZrgqO2OaNCxcGC3vj0RmCrxr0oEqFwVxvjqBtKBQKfPomFGpJLxyRsbC0ZGRPLqN1AOM-46YjwxW2VNYQIYX4J__iSOZOKWZSfWEDtkAzNGPmzIfItw4QCRDDexNqgQ=11642C6F'

projects_url = f"{JIRA_URL}/rest/api/2/project"
issues_url = f"{JIRA_URL}/rest/api/2/search"
issue_url = f"{JIRA_URL}/rest/api/2/issue/{{}}"
attachment_url = f"{JIRA_URL}/secure/attachment/{{}}/{{}}"

auth = (USERNAME, API_TOKEN)

params = {
    'fields': 'key',  
    'maxResults': 50, 
    'startAt': 0 
}

allowed_file_types = [
    'log', 'txt', 'json', 'xml', 'yaml', 'java', 'patch', 'csv', 'xls', 'xlsx', 'pdf', 'zip', 'rar', 'docx'
]

output_directory = 'attachments'
os.makedirs(output_directory, exist_ok=True)

response = requests.get(projects_url, auth=auth)
response.raise_for_status()
projects_data = response.json()

for project_data in projects_data:
    project_key = project_data['key']
    project_name = project_data['name']
    project_directory = os.path.join(output_directory, project_key)
    os.makedirs(project_directory, exist_ok=True)

    print(f"Processing project: {project_name} ({project_key})")

    params['jql'] = f'project={project_key}'

    while True:
        response = requests.get(issues_url, auth=auth, params=params)
        response.raise_for_status()

        data = response.json()

        issues = data.get('issues', [])

        if not issues:
            break

        for issue in issues:
            issue_key = issue['key']
            issue_directory = os.path.join(project_directory, issue_key)
            os.makedirs(issue_directory, exist_ok=True)

            print(f"Processing issue: {issue_key}")

            response = requests.get(issue_url.format(issue_key), auth=auth)
            response.raise_for_status()
            issue_details = response.json()

            attachments = issue_details.get('fields', {}).get('attachment', [])

            supported_attachments_found = False

            for attachment in attachments:
                attachment_id = attachment.get('id')
                attachment_name = attachment.get('filename')
                attachment_file_type = attachment_name.split('.')[-1].lower() if attachment_name else None

                if attachment_id and attachment_name:
                    if attachment_file_type in allowed_file_types:
                        attachment_response = requests.get(attachment_url.format(attachment_id, attachment_name), auth=auth)

                        if attachment_response.status_code == 200:
                            output_path = os.path.join(issue_directory, attachment_name)
                            with open(output_path, 'wb') as f:
                                f.write(attachment_response.content)
                                supported_attachments_found = True
                        else:
                            print(f"Failed to download attachment for issue {issue_key}, attachment ID {attachment_id}, status code: {attachment_response.status_code}")
                    else:
                        print(f"Skipping unsupported attachment for issue {issue_key}: {attachment_name} (Type: {attachment_file_type})")

            if not supported_attachments_found:
                print(f"No supported attachments found for issue {issue_key}")

        params['startAt'] += params['maxResults']

    params['startAt'] = 0

print("All projects processed.")

Processing project: Hibernate Commons Annotations (HCANN)
Processing issue: HCANN-134
No supported attachments found for issue HCANN-134
Processing issue: HCANN-133
No supported attachments found for issue HCANN-133
Processing issue: HCANN-132
No supported attachments found for issue HCANN-132
Processing issue: HCANN-131
No supported attachments found for issue HCANN-131
Processing issue: HCANN-129
No supported attachments found for issue HCANN-129
Processing issue: HCANN-128
No supported attachments found for issue HCANN-128
Processing issue: HCANN-127
No supported attachments found for issue HCANN-127
Processing issue: HCANN-126
No supported attachments found for issue HCANN-126
Processing issue: HCANN-125
No supported attachments found for issue HCANN-125
Processing issue: HCANN-124
No supported attachments found for issue HCANN-124
Processing issue: HCANN-123
No supported attachments found for issue HCANN-123
Processing issue: HCANN-122
No supported attachments found for issue HCAN

No supported attachments found for issue HCANN-20
Processing issue: HCANN-19
No supported attachments found for issue HCANN-19
Processing issue: HCANN-18
No supported attachments found for issue HCANN-18
Processing issue: HCANN-6
No supported attachments found for issue HCANN-6
Processing issue: HCANN-5
No supported attachments found for issue HCANN-5
Processing issue: HCANN-4
No supported attachments found for issue HCANN-4
Processing issue: HCANN-3
No supported attachments found for issue HCANN-3
Processing issue: HCANN-2
No supported attachments found for issue HCANN-2
Processing issue: HCANN-1
No supported attachments found for issue HCANN-1
Processing project: z - Hibernate Annotations (ANN)
Processing issue: ANN-813
Skipping unsupported attachment for issue ANN-813: ANN-813.diff (Type: diff)
No supported attachments found for issue ANN-813
Processing issue: ANN-805
No supported attachments found for issue ANN-805
Processing issue: ANN-803
No supported attachments found for issue 

No supported attachments found for issue ANN-570
Processing issue: ANN-569
No supported attachments found for issue ANN-569
Processing issue: ANN-567
Processing issue: ANN-566
Processing issue: ANN-563
No supported attachments found for issue ANN-563
Processing issue: ANN-562
No supported attachments found for issue ANN-562
Processing issue: ANN-561
No supported attachments found for issue ANN-561
Processing issue: ANN-560
No supported attachments found for issue ANN-560
Processing issue: ANN-559
No supported attachments found for issue ANN-559
Processing issue: ANN-558
No supported attachments found for issue ANN-558
Processing issue: ANN-557
No supported attachments found for issue ANN-557
Processing issue: ANN-556
No supported attachments found for issue ANN-556
Processing issue: ANN-555
No supported attachments found for issue ANN-555
Processing issue: ANN-554
No supported attachments found for issue ANN-554
Processing issue: ANN-553
No supported attachments found for issue ANN-553

No supported attachments found for issue ANN-398
Processing issue: ANN-397
No supported attachments found for issue ANN-397
Processing issue: ANN-396
No supported attachments found for issue ANN-396
Processing issue: ANN-395
No supported attachments found for issue ANN-395
Processing issue: ANN-393
No supported attachments found for issue ANN-393
Processing issue: ANN-392
No supported attachments found for issue ANN-392
Processing issue: ANN-390
No supported attachments found for issue ANN-390
Processing issue: ANN-389
No supported attachments found for issue ANN-389
Processing issue: ANN-388
No supported attachments found for issue ANN-388
Processing issue: ANN-387
No supported attachments found for issue ANN-387
Processing issue: ANN-386
No supported attachments found for issue ANN-386
Processing issue: ANN-385
No supported attachments found for issue ANN-385
Processing issue: ANN-384
No supported attachments found for issue ANN-384
Processing issue: ANN-383
No supported attachments 

No supported attachments found for issue ANN-267
Processing issue: ANN-266
No supported attachments found for issue ANN-266
Processing issue: ANN-265
No supported attachments found for issue ANN-265
Processing issue: ANN-264
Processing issue: ANN-263
No supported attachments found for issue ANN-263
Processing issue: ANN-262
No supported attachments found for issue ANN-262
Processing issue: ANN-261
Processing issue: ANN-259
Processing issue: ANN-258
No supported attachments found for issue ANN-258
Processing issue: ANN-257
Processing issue: ANN-256
No supported attachments found for issue ANN-256
Processing issue: ANN-255
Processing issue: ANN-254
No supported attachments found for issue ANN-254
Processing issue: ANN-253
Processing issue: ANN-252
No supported attachments found for issue ANN-252
Processing issue: ANN-251
No supported attachments found for issue ANN-251
Processing issue: ANN-249
No supported attachments found for issue ANN-249
Processing issue: ANN-248
No supported attach

...

No supported attachments found for issue HBX-301
Processing issue: HBX-300
No supported attachments found for issue HBX-300
Processing issue: HBX-299
No supported attachments found for issue HBX-299
Processing issue: HBX-298
No supported attachments found for issue HBX-298
Processing issue: HBX-297
No supported attachments found for issue HBX-297
Processing issue: HBX-295
No supported attachments found for issue HBX-295
Processing issue: HBX-294
No supported attachments found for issue HBX-294
Processing issue: HBX-293
No supported attachments found for issue HBX-293
Processing issue: HBX-292
No supported attachments found for issue HBX-292
Processing issue: HBX-291
No supported attachments found for issue HBX-291
Processing issue: HBX-290
No supported attachments found for issue HBX-290
Processing issue: HBX-289
No supported attachments found for issue HBX-289
Processing issue: HBX-288
No supported attachments found for issue HBX-288
Processing issue: HBX-287
No supported attachments 

No supported attachments found for issue HBX-153
Processing issue: HBX-152
Processing issue: HBX-151
Processing issue: HBX-150
No supported attachments found for issue HBX-150
Processing issue: HBX-149
No supported attachments found for issue HBX-149
Processing issue: HBX-148
No supported attachments found for issue HBX-148
Processing issue: HBX-147
No supported attachments found for issue HBX-147
Processing issue: HBX-146
No supported attachments found for issue HBX-146
Processing issue: HBX-145
No supported attachments found for issue HBX-145
Processing issue: HBX-144
No supported attachments found for issue HBX-144
Processing issue: HBX-143
No supported attachments found for issue HBX-143
Processing issue: HBX-142
No supported attachments found for issue HBX-142
Processing issue: HBX-141
No supported attachments found for issue HBX-141
Processing issue: HBX-140
Processing issue: HBX-139
Skipping unsupported attachment for issue HBX-139: db-schema.sql (Type: sql)
No supported attachm

Processing issue: HBX-34
No supported attachments found for issue HBX-34
Processing issue: HBX-33
No supported attachments found for issue HBX-33
Processing issue: HBX-32
Skipping unsupported attachment for issue HBX-32: patchHbm2JavaTask.diff (Type: diff)
No supported attachments found for issue HBX-32
Processing issue: HBX-31
No supported attachments found for issue HBX-31
Processing issue: HBX-30
No supported attachments found for issue HBX-30
Processing issue: HBX-29
No supported attachments found for issue HBX-29
Processing issue: HBX-28
Skipping unsupported attachment for issue HBX-28: patchJavadoc.diff (Type: diff)
Skipping unsupported attachment for issue HBX-28: patchJavadoc.diff (Type: diff)
No supported attachments found for issue HBX-28
Processing issue: HBX-27
Processing issue: HBX-26
No supported attachments found for issue HBX-26
Processing issue: HBX-25
No supported attachments found for issue HBX-25
Processing issue: HBX-24
No supported attachments found for issue HBX-

No supported attachments found for issue HHH-18601
Processing issue: HHH-18600
No supported attachments found for issue HHH-18600
Processing issue: HHH-18599
No supported attachments found for issue HHH-18599
Processing issue: HHH-18598
No supported attachments found for issue HHH-18598
Processing issue: HHH-18597
No supported attachments found for issue HHH-18597
Processing issue: HHH-18596
No supported attachments found for issue HHH-18596
Processing issue: HHH-18595
No supported attachments found for issue HHH-18595
Processing issue: HHH-18594
No supported attachments found for issue HHH-18594
Processing issue: HHH-18593
No supported attachments found for issue HHH-18593
Processing issue: HHH-18592
No supported attachments found for issue HHH-18592
Processing issue: HHH-18591
No supported attachments found for issue HHH-18591
Processing issue: HHH-18590
No supported attachments found for issue HHH-18590
Processing issue: HHH-18589
No supported attachments found for issue HHH-18589
P

HTTPError: 503 Server Error: Service Unavailable for url: https://hibernate.atlassian.net/rest/api/2/issue/HHH-18558